In [1]:
import os
import sys
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', None, '')
FLAGS(sys.argv)
from projects.feed.rank.src.evaluate import evaluate_df as eval
import pymp
from multiprocessing import Pool, Manager, cpu_count 
import qgrid

tensorflow_version: 1.14.0
torch_version: 1.3.1+cu100
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
mark = 'video'
root = f'/search/odin/publicData/CloudS/chenghuige/rank/data/{mark}_hour_sgsapp_v1/exps/monitor'
root = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v1/exps/monitor'
day_ = 20191226
days = 100
day = gezi.DateTime(str(day_), day=True)

In [3]:
models = [os.path.basename(x) for x in glob.glob(f'{root}/*')]
# names = [
#   'fm',
#   'fm.deeponly',
# ]
models

['fm',
 'base',
 'fm.1st',
 'fm.val',
 'fm.no0',
 'fm.val.no0',
 'fm.val.hashv2',
 'fm.val.hashv2.ud',
 'baseline']

In [4]:
dfs = Manager().list()
files = []
for model in models:
  files += glob.glob(f'{root}/{model}/infos_day/*/valid.csv')
ps = min(len(files), cpu_count())
with pymp.Parallel(ps) as p:
  for i in tqdm(p.range(len(files))):
    dfs += [pd.read_csv(files[i])]
dfs = list(dfs)

100%|██████████| 8/8 [00:00<00:00, 20.03it/s]


 75%|███████▌  | 6/8 [00:00<00:00, 22.20it/s]

 50%|█████     | 4/8 [00:00<00:00, 17.35it/s]




100%|██████████| 8/8 [00:00<00:00, 28.23it/s]

100%|██████████| 7/7 [00:00<00:00, 27.41it/s]

100%|██████████| 8/8 [00:00<00:00, 27.75it/s]


In [5]:
df = pd.concat(dfs)

In [6]:
df[df.model=='baseline'].sort_values('date')[['date', 'group/ndcg_click']].tail()

,date,group/ndcg_click
0,20200127,0.589593
0,20200128,0.590548
0,20200129,0.592920
0,20200130,0.589560
0,20200131,0.584201


In [7]:
df['sqrt/auc'] = (df['auc'] * df['click/time_auc']) ** 0.5
df['group/sqrt/auc'] =  (df['group/auc'] * df['group/click/time_auc']) ** 0.5
df['quality/sqrt/auc'] = (df['quality/auc'] * df['quality/click/time_auc']) ** 0.5
df['quality/group/sqrt/auc'] =  (df['quality/group/auc'] * df['quality/group/click/time_auc']) ** 0.5

In [8]:
cols = ['date', 'model','sqrt/auc', 'group/sqrt/auc', 'quality/sqrt/auc', 'quality/group/sqrt/auc', 'inv_rate']
if mark == 'video':
  cols += ['finish_auc', 'vtime_auc']
qgrid.show_grid(df[cols].sort_values(['date', 'group/sqrt/auc'], ascending=[False, False]))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [9]:
# df[['date', 'model', 'auc', 'group/auc', 'cold/auc', 'clickmids/auc',
#     'click/time_auc','group/click/time_auc',
#    ]].sort_values(['date', 'group/auc'], ascending=[False, False]).head(10)

In [10]:
# qgrid.show_grid(df[[
#     'date', 'model','quality/auc', 'quality/group/auc', 
#     'quality/click/time_auc', 'quality/group/click/time_auc'
# ]].sort_values(['date', 'quality/click/time_auc'], ascending=[False, False]))

In [11]:
# df[[
#       'date', 'model', 'group/click/top1_dur_rate', 'group/click/top3_dur_rate', 'group/top1_click_rate', 'group/top3_click_rate'
# ]].sort_values(['date', 'group/top3_click_rate'], ascending=[False, False]).head(10)

In [12]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:500000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [13]:
df = df[df.hours > 20]

In [14]:
metrics = ['hours', 'group/sqrt/auc', 'sqrt/auc', 
           'group/auc', 'group/auc2', 'auc', 'clickmids/auc', 'cold/auc',
           'group/click/time_auc', 'quality/group/click/time_auc',
           'finish_auc', 'rcr_auc', 'vtime_auc', 'ptime_auc']
gezi.line(df, x='date', y=metrics, color='model')

In [15]:
metrics = [
           'group/sqrt/auc', 'sqrt/auc', 
           'group/click/time_auc',
           'quality/group/click/time_auc', 'quality/group/auc',
           'group/top1_click_rate', 'group/top3_click_rate',
           'group/click/top1_dur_rate', 'group/click/top3_dur_rate',
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [16]:
metrics = [
  'group/first_click_position',
  'group/last_click_position',
  'group/ndcg_click',
  'group/ndcg3_click',
  'group/ndcg3_dur',
  'group/ndcg_dur',
#   'group/ndcg7_click',
#   'group/ndcg14_click',
  'group/click/ndcg_dur',
  'group/click/ndcg3_dur',
#   'group/click/ndcg7_dur',
#   'group/click/ndcg14_dur',
  'quality/group/ndcg_dur',
  'quality/group/click/ndcg_dur',
  'quality/group/click/ndcg3_dur',
]
gezi.line(df, x='date', y=metrics, color='model')